In [98]:
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import os
import time
import requests

# Load environment variables
load_dotenv()

True

In [99]:
# === Список проектов с соответствующими переменными среды ===
projects = [
    {
        "name": "Ozon-A&D Rus-1112223",
        "client_id": os.getenv("IdAnd_Ozon"),
        "api_key": os.getenv("KeyAnd_Ozon")
    },
    {
        "name": "Ozon-Braun Russia-19885",
        "client_id": os.getenv("IdBraun_Ozon"),
        "api_key": os.getenv("KeyBraun_Ozon")
    },
    {
        "name": "Ozon-CASO-100304",
        "client_id": os.getenv("IdCaso_Ozon"),
        "api_key": os.getenv("KeyCaso_Ozon")
    },
    {
        "name": "Ozon-Gillette-Club-80466",
        "client_id": os.getenv("IdGillette_Ozon"),
        "api_key": os.getenv("KeyGillette_Ozon")
    },
    {
        "name": "Ozon-GUTENTECH-1547",
        "client_id": os.getenv("IdGuten_Ozon"),
        "api_key": os.getenv("KeyGuten_Ozon")
    },
    {
        "name": "Ozon-KitchenAid-1638",
        "client_id": os.getenv("IdKitchen_Ozon"),
        "api_key": os.getenv("KeyKitchen_Ozon")
    },
    {
        "name": "Ozon-Smart Market-1676213",
        "client_id": os.getenv("IdSmart_Ozon"),
        "api_key": os.getenv("KeySmart_Ozon")
    }
]

# === URL API ===
url = "https://api-seller.ozon.ru/v3/finance/transaction/list" 

# === Фильтр по дате: последние 2 дня ===
end_date = datetime.today() - timedelta(days=1)
start_date = end_date - timedelta(days=30)

# === Сюда будем собирать данные со всех проектов ===
all_data = []

for project in projects:
    print(f"Получаем данные для проекта: {project['name']}")

    client_id = project["client_id"]
    api_key = project["api_key"]

    if not client_id or not api_key:
        print(f"Не указаны Client-ID или API-Key для проекта: {project['name']}")
        continue

    headers = {
        "Client-Id": client_id,
        "Api-Key": api_key,
        "Content-Type": "application/json"
    }

    body = {
        "filter": {
            "date": {
                "from": start_date.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
                "to": end_date.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "operation_type": [],
            "posting_number": "",
            "transaction_type": "all"
        },
        "page": 1,
        "page_size": 1000
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))

    if response.status_code == 200:
        print(f"Данные получены для {project['name']}")
        operations = response.json().get("result", {}).get("operations", [])

        if not operations:
            print(f"Нет данных за указанный период для {project['name']}")
            continue

        # === Подготовка данных ===
        rows = []
        for op in operations:
            base = {
                'operation_id': op.get('operation_id'),
                'operation_type': op.get('operation_type'),
                'operation_date': op.get('operation_date'),
                'operation_type_name': op.get('operation_type_name'),
                'delivery_charge': op.get('delivery_charge'),
                'return_delivery_charge': op.get('return_delivery_charge'),
                'accruals_for_sale': op.get('accruals_for_sale'),
                'sale_commission': op.get('sale_commission'),
                'amount': op.get('amount'),
                'type': op.get('type')
            }

            posting = op.get('posting', {})
            base.update({
                'posting_delivery_schema': posting.get('delivery_schema'),
                'posting_order_date': posting.get('order_date'),
                'posting_posting_number': posting.get('posting_number'),
                'posting_warehouse_id': posting.get('warehouse_id')
            })

            items = op.get('items', [])
            if items:
                item = items[0]
                base.update({
                    'item_name': item.get('name'),
                    'item_sku': item.get('sku')
                })
            else:
                base.update({'item_name': None, 'item_sku': None})

            services = op.get('services', [])
            if services:
                service = services[0]
                base.update({
                    'service_name': service.get('name'),
                    'service_price': service.get('price')
                })
            else:
                base.update({'service_name': None, 'service_price': None})

            # Добавляем имя проекта
            base['project'] = project['name']

            rows.append(base)

        all_data.extend(rows)

    else:
        print(f"Ошибка при запросе к Ozon API для {project['name']}: {response.status_code}")
        print(response.text)

# === Создание общего DataFrame ===
if all_data:
    df = pd.DataFrame(all_data)
    df = df.where(pd.notnull(df), None)

    # === Явное приведение типов под схему ClickHouse ===
    df['operation_id'] = pd.to_numeric(df['operation_id'], errors='coerce').fillna(0).astype('int64')
    df['delivery_charge'] = pd.to_numeric(df['delivery_charge'], errors='coerce').fillna(0).astype('float64')
    df['return_delivery_charge'] = pd.to_numeric(df['return_delivery_charge'], errors='coerce').fillna(0).astype('float64')
    df['accruals_for_sale'] = pd.to_numeric(df['accruals_for_sale'], errors='coerce').fillna(0).astype('float64')
    df['sale_commission'] = pd.to_numeric(df['sale_commission'], errors='coerce').fillna(0).astype('float64')
    df['amount'] = pd.to_numeric(df['amount'], errors='coerce').fillna(0).astype('float64')
    df['item_sku'] = pd.to_numeric(df['item_sku'], errors='coerce').fillna(0).astype('int64')

    # === Преобразование дат в datetime и замена NaT на None ===
    df['operation_date'] = pd.to_datetime(df['operation_date'], errors='coerce')
    df['posting_order_date'] = pd.to_datetime(df['posting_order_date'], errors='coerce')

    # Замена NaT на None для Nullable(DateTime) полей
    df['operation_date'] = df['operation_date'].where(df['operation_date'].notnull(), None)
    df['posting_order_date'] = df['posting_order_date'].where(df['posting_order_date'].notnull(), None)


else:
    print("Нет данных для отображения")

Получаем данные для проекта: Ozon-A&D Rus-1112223
Данные получены для Ozon-A&D Rus-1112223
Получаем данные для проекта: Ozon-Braun Russia-19885
Данные получены для Ozon-Braun Russia-19885
Получаем данные для проекта: Ozon-CASO-100304
Данные получены для Ozon-CASO-100304
Получаем данные для проекта: Ozon-Gillette-Club-80466
Данные получены для Ozon-Gillette-Club-80466
Получаем данные для проекта: Ozon-GUTENTECH-1547
Данные получены для Ozon-GUTENTECH-1547
Получаем данные для проекта: Ozon-KitchenAid-1638
Данные получены для Ozon-KitchenAid-1638
Получаем данные для проекта: Ozon-Smart Market-1676213
Данные получены для Ozon-Smart Market-1676213


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   operation_id             5500 non-null   int64         
 1   operation_type           5500 non-null   object        
 2   operation_date           5500 non-null   datetime64[ns]
 3   operation_type_name      5500 non-null   object        
 4   delivery_charge          5500 non-null   float64       
 5   return_delivery_charge   5500 non-null   float64       
 6   accruals_for_sale        5500 non-null   float64       
 7   sale_commission          5500 non-null   float64       
 8   amount                   5500 non-null   float64       
 9   type                     5500 non-null   object        
 10  posting_delivery_schema  5500 non-null   object        
 11  posting_order_date       5250 non-null   datetime64[ns]
 12  posting_posting_number   5500 non-

In [101]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [102]:
# Ensure the DataFrame has the correct columns
columns = ['project','operation_id', 'operation_type', 'operation_date',
       'operation_type_name', 'delivery_charge', 'return_delivery_charge',
       'accruals_for_sale', 'sale_commission', 'amount', 'type',
       'posting_delivery_schema', 'posting_order_date',
       'posting_posting_number', 'posting_warehouse_id', 'item_name',
       'item_sku', 'service_name', 'service_price']

# Reorder columns to match the expected order
df_modified = df[columns]
# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_modified.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'ozon_finance'
# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Error serializing column `posting_order_date` into data type `Nullable(DateTime)`
Traceback (most recent call last):
  File "c:\Users\User\Desktop\Data\.venv\Lib\site-packages\clickhouse_connect\driver\transform.py", line 99, in chunk_gen
    col_type.write_column(data, output, context)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\Data\.venv\Lib\site-packages\clickhouse_connect\datatypes\base.py", line 214, in write_column
    self.write_column_data(column, dest, ctx)
    ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\Data\.venv\Lib\site-packages\clickhouse_connect\datatypes\base.py", line 229, in write_column_data
    self._write_column_binary(column, dest, ctx)
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\Data\.venv\Lib\site-packages\clickhouse_connect\datatypes\temporal.py", line 132, in _write_column_binary
    column = [int(x.timestamp()) if x else 0 for x in column]
                  ~~~~

Sample data to insert: [('Ozon-A&D Rus-1112223', 32479221362, 'OperationAgentDeliveredToCustomer', Timestamp('2025-05-10 00:00:00'), 'Доставка покупателю', 0.0, 0.0, 4811.0, -962.2, 3379.88, 'orders', 'FBO', Timestamp('2025-05-07 22:50:18'), '86937412-0035-1', 1020000890160000, 'Массажер антицеллюлитный медицинский электрический Nozomi MH-102 с лампой инфракрасного излучения, массаж для шеи и плеч, суставов и тела', 959010529, 'MarketplaceServiceItemDelivToCustomer', -215.04), ('Ozon-A&D Rus-1112223', 32479283941, 'MarketplaceRedistributionOfAcquiringOperation', Timestamp('2025-05-10 00:00:00'), 'Оплата эквайринга', 0.0, 0.0, 0.0, 0.0, -10.79, 'other', '', Timestamp('2025-05-10 05:20:57'), '50116110-0084', 0, 'Груша (нагнетатель) AND RB-101G', 1056477332, 'MarketplaceRedistributionOfAcquiringOperation', -10.79), ('Ozon-A&D Rus-1112223', 32479976826, 'MarketplaceRedistributionOfAcquiringOperation', Timestamp('2025-05-10 00:00:00'), 'Оплата эквайринга', 0.0, 0.0, 0.0, 0.0, -38.36, 'other

ValueError: NaTType does not support timestamp